In [1]:
import pandas as pd

In [2]:
#import annotazioni e gold standard
annotazioni= pd.read_csv('../files/annotazioni_full.csv', delimiter=',')
GS= pd.read_csv('../files/gold_standard.csv', delimiter=',')

# descrizione problema

In [3]:
annotazioni.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'annotator'],
      dtype='object')

In [4]:
GS.columns

Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui'],
      dtype='object')

In [5]:
#allineazione nomi
annotazioni = annotazioni.rename(columns={'ontologies':'Ontologies','SOURCE':'Source','entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})
annotazioni['Ontologies'] = annotazioni['Ontologies'].replace('SNOMEDCT', 'SNOMED-CT')

GS = GS.rename(columns={'entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})

In [6]:

set_gold = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_full = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Calcoliamo i valori richiesti
Nboth = len(set_gold & set_full)  # Presenti in entrambi
Nleft = len(set_gold - set_full)  # Solo in df_gold
Nright = len(set_full - set_gold)  # Solo in df_full

print(f"Nboth: {Nboth}, Nleft: {Nleft}, Nright: {Nright}")

Nboth: 58, Nleft: 34, Nright: 1345


In [7]:
#dettagliare questa analisi rispetto alle diverse ontologie

set_GS = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_annotazioni = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Unione dei due DataFrame per analisi per Ontology
df_combined = pd.concat([GS.assign(source='GS'), annotazioni.assign(source='annotazioni')])

# Creiamo una tabella che conta Nleft, Nright e Nboth per ogni Ontology
ontology_counts = df_combined.groupby(['Ontologies', 'source']).size().unstack(fill_value=0)
ontology_counts.columns = ['Nleft', 'Nright']
ontology_counts['Nboth'] = ontology_counts.min(axis=1)  # Il minimo tra Nleft e Nright dà Nboth

# Output finale
print(ontology_counts)

            Nleft  Nright  Nboth
Ontologies                      
LOINC           0    7903      0
SNOMED-CT     127   20398    127
UMLS          196    9801    196


In [8]:
#dettagliare questa analisi rispetto alle diverse annotazioni
# Aggiungiamo un annotatore fittizio per GS
GS['Annotator'] = 'Gold_Standard'

# Cambiamo il nome della colonna 'source' in 'Provenienza'
GS['Provenienza'] = 'GS'
annotazioni['Provenienza'] = 'annotazioni'
df_combined = pd.concat([GS, annotazioni])

# Raggruppiamo per Ontology e Annotator
ontology_annotator_counts = df_combined.groupby(['Ontologies', 'Annotator', 'Provenienza']).size().unstack(fill_value=0)

# Rinominiamo le colonne
ontology_annotator_counts.columns = ['Nleft', 'Nright']
ontology_annotator_counts['Nboth'] = ontology_annotator_counts.min(axis=1)

# Output finale
print(ontology_annotator_counts)

                          Nleft  Nright  Nboth
Ontologies Annotator                          
LOINC      Bioportal          0    7903      0
SNOMED-CT  Bioportal          0    4791      0
           Gold_Standard    127       0      0
           MedCAT             0   15607      0
UMLS       Gold_Standard    196       0      0
           MedCAT             0    9801      0


# Valutazione della Qualità degli Annotatori

## Metriche di valutazione

In [12]:
def evaluate_annotation(gs_df, ann_df, column):

    gs = set(gs_df[column].dropna())
    ann = set(ann_df[column].dropna())

    true_positives = len(gs & ann)  # Elementi corretti
    precision = true_positives / len(ann) if len(ann) > 0 else 0
    recall = true_positives / len(gs) if len(gs) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1-score": round(f1_score, 4)
    }

# Esempio di utilizzo


metrics = evaluate_annotation(GS, annotazioni, "AttributoNomeCompleto")
print(metrics)

{'Precision': 0.0576, 'Recall': 0.5541, 'F1-score': 0.1043}


## Metodi di confronto

In [17]:
def metodi_confronto(gs_df, ann_df, column):

    gs = set(gs_df[column].dropna())
    ann = set(ann_df[column].dropna())

    intersection = gs & ann  # Concetti comuni

    # Jaccard Similarity
    jaccard = len(intersection) / len(gs | ann) if len(gs | ann) > 0 else 0

    # Weighted Overlap
    weighted_overlap = sum(1 for concept in intersection) / sum(1 for concept in gs) if len(gs) > 0 else 0

    return {
        "Intersection": len(intersection),
        "Jaccard Similarity": round(jaccard, 4),
        "Weighted Overlap": round(weighted_overlap, 4),
        "Valori dell'intersezione": intersection
    }



metrics = metodi_confronto(GS, annotazioni, "AttributoNomeCompleto")
print(metrics)

{'Jaccard Similarity': 0.055, 'Weighted Overlap': 0.5541, 'Intersection': 41, "Valori dell'intersezione": {'person-gender_source_concept_id', 'condition_occurrence-provider_id', 'condition_occurrence-condition_source_concept_id', 'person-ethnicity_source_value', 'condition_occurrence-condition_type_concept_id', 'location-zip', 'location-address_1', 'condition_occurrence-condition_status_source_value', 'person-year_of_birth', 'location-address_2', 'condition_occurrence-stop_reason', 'person-race_source_concept_id', 'person-gender_source_value', 'condition_occurrence-person_id', 'location-county', 'person-person_id', 'condition_occurrence-condition_concept_id', 'location-city', 'person-ethnicity_concept_id', 'person-month_of_birth', 'location-location_id', 'location-state', 'location-location_source_value', 'condition_occurrence-condition_occurrence_id', 'person-ethnicity_source_concept_id', 'condition_occurrence-visit_detail_id', 'person-race_source_value', 'condition_occurrence-conditi

In [27]:
gs_df = GS.assign(source='GS')
# Calcoliamo la frequenza d'uso dei concetti nel Gold Standard
gs_freq = gs_df['AttributoNomeCompleto'].value_counts()


# Mappiamo la frequenza come peso nei concetti di GS
gs_df['Peso'] = gs_df['AttributoNomeCompleto'].map(gs_freq)


# Uniamo i due DataFrame per ottenere i concetti comuni
merged_df = pd.merge(gs_df, annotazioni, on='AttributoNomeCompleto', how='inner')

# Calcoliamo l'overlap ponderato sommando i pesi dei concetti comuni
overlap_ponderato = (merged_df['Peso'] * gs_freq[merged_df['AttributoNomeCompleto']]).sum()

print(f'Overlap Ponderato: {overlap_ponderato}')


Overlap Ponderato: 0.0


NON CAPISCO PERCHÈ L'OVERLAP PONDERATO SIA UGUALE A 0

In [29]:
#Troviamo l'intersezione tra Gold Standard e Annotazioni
common_concepts = set(GS['AttributoNomeCompleto']).intersection(set(annotazioni['AttributoNomeCompleto']))

# 3. Calcoliamo la percentuale di concetti del Gold Standard presenti nelle Annotazioni
total_concepts_gs = len(GS)
correctly_annotated = len(common_concepts)

# Percentuale di concetti correttamente annotati
annotation_accuracy = (correctly_annotated / total_concepts_gs) * 100

# 4. Valutiamo la bontà degli annotatori
print(f'Concetti totali nel Gold Standard: {total_concepts_gs}')
print(f'Concetti correttamente annotati: {correctly_annotated}')
print(f'Accuratezza delle annotazioni: {annotation_accuracy:.2f}%')

Concetti totali nel Gold Standard: 323
Concetti correttamente annotati: 41
Accuratezza delle annotazioni: 12.69%


## Calcolo metriche su più attributi

In [32]:
def calculate_metrics(gs_df, ann_df):
    # Inizializziamo un dizionario per memorizzare i risultati per ogni concetto
    metrics = {'AttributoNomeCompleto': [], 'Precision': [], 'Recall': [], 'F1': []}

    # Elenco dei concetti unici presenti nel GS
    all_concepts = gs_df['AttributoNomeCompleto'].unique()

    for concept in all_concepts:
        # Filtra i concetti corrispondenti nel GS e nelle Annotazioni
        gs_concepts = set(gs_df[gs_df['AttributoNomeCompleto'] == concept]['AttributoNomeCompleto'])
        ann_concepts = set(ann_df[ann_df['AttributoNomeCompleto'] == concept]['AttributoNomeCompleto'])

        # Calcolo della Precision
        precision = len(gs_concepts.intersection(ann_concepts)) / len(ann_concepts) if len(ann_concepts) > 0 else 0

        # Calcolo del Recall
        recall = len(gs_concepts.intersection(ann_concepts)) / len(gs_concepts) if len(gs_concepts) > 0 else 0

        # Calcolo del F1-Score
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Memorizza i risultati per ogni concetto
        metrics['AttributoNomeCompleto'].append(concept)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1'].append(f1)

    # Convertiamo in DataFrame per una migliore visualizzazione
    metrics_df = pd.DataFrame(metrics)

    return metrics_df

def calculate_aggregated_metrics(metrics_df):
    # Macro-average: media delle metriche
    precision_macro = metrics_df['Precision'].mean()
    recall_macro = metrics_df['Recall'].mean()
    f1_macro = metrics_df['F1'].mean()

    # Micro-average: somma dei veri positivi, falsi positivi e falsi negativi
    total_tp = sum([metrics_df.iloc[i]['Precision'] * metrics_df.iloc[i]['Recall'] for i in range(len(metrics_df))])
    total_fp = sum([metrics_df.iloc[i]['Precision'] * (1 - metrics_df.iloc[i]['Recall']) for i in range(len(metrics_df))])
    total_fn = sum([metrics_df.iloc[i]['Recall'] * (1 - metrics_df.iloc[i]['Precision']) for i in range(len(metrics_df))])

    precision_micro = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall_micro = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_micro = (2 * precision_micro * recall_micro) / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    # Restituisce le metriche aggregate
    aggregated_metrics = {
        'Precisionmacro': precision_macro,
        'Recallmacro': recall_macro,
        'F1macro': f1_macro,
        'Precisionmicro': precision_micro,
        'Recallmicro': recall_micro,
        'F1micro': f1_micro
    }

    return aggregated_metrics

# Calcoliamo le metriche per ciascun concetto (AttributoNomeCompleto)
metrics_df = calculate_metrics(gs_df, annotazioni)

# Calcoliamo le metriche aggregate (Macro e Micro)
aggregated_metrics = calculate_aggregated_metrics(metrics_df)

# Visualizziamo le metriche aggregate
print("Metriche Aggregate:")
for metric, value in aggregated_metrics.items():
    print(f"{metric}: {value}")


Metriche Aggregate:
Precisionmacro: 0.5540540540540541
Recallmacro: 0.5540540540540541
F1macro: 0.5540540540540541
Precisionmicro: 1.0
Recallmicro: 1.0
F1micro: 1.0


## Analisi della Similarit`a tra Attributi con Annotazioni Diverse

In [35]:
import pandas as pd

# Funzione per calcolare l'intersezione tra due insiemi
def intersection(set1, set2):
    return len(set1.intersection(set2))

# Funzione per calcolare la similarità di Jaccard
def jaccard_similarity(set1, set2):
    return len(set1.intersection(set2)) / len(set1.union(set2)) if len(set1.union(set2)) > 0 else 0

# Funzione per calcolare il coefficiente di Sorensen-Dice
def sorensen_dice(set1, set2):
    return (2 * len(set1.intersection(set2))) / (len(set1) + len(set2)) if (len(set1) + len(set2)) > 0 else 0

# Funzione per calcolare la corrispondenza per ogni attributo
def calculate_similarity(gs_df, ann_df, threshold):
    similarity_results = {
        'AttributoNomeCompleto': [],
        'Jaccard': [],
        'Sorensen-Dice': [],
        'Corrispondenza': []
    }

    # Iteriamo su ciascun attributo
    for attribute in gs_df['AttributoNomeCompleto'].unique():
        # Prendiamo gli insiemi di annotazioni per l'attributo in entrambi gli schemi
        gs_annotations = set(gs_df[gs_df['AttributoNomeCompleto'] == attribute]['PrettyName'])
        ann_annotations = set(ann_df[ann_df['AttributoNomeCompleto'] == attribute]['PrettyName'])

        # Calcoliamo le metriche di similarità
        jaccard = jaccard_similarity(gs_annotations, ann_annotations)
        dice = sorensen_dice(gs_annotations, ann_annotations)

        # Verifica della corrispondenza (Jaccard)
        corrispondenza = 1 if jaccard >= threshold else 0

        # Aggiungiamo i risultati
        similarity_results['AttributoNomeCompleto'].append(attribute)
        similarity_results['Jaccard'].append(jaccard)
        similarity_results['Sorensen-Dice'].append(dice)
        similarity_results['Corrispondenza'].append(corrispondenza)

    # Creiamo un DataFrame con i risultati
    similarity_df = pd.DataFrame(similarity_results)

    return similarity_df



# Definiamo una soglia di similarità per la corrispondenza
threshold = 0.8

# Calcoliamo la similarità tra gli attributi
similarity_df = calculate_similarity(GS, annotazioni, threshold)

# Visualizziamo i risultati
print(similarity_df)


                 AttributoNomeCompleto   Jaccard  Sorensen-Dice  \
0                    location-location  0.000000       0.000000   
1                 location-location_id  0.043478       0.083333   
2                   location-address_1  0.040000       0.076923   
3                   location-address_2  0.038462       0.074074   
4                        location-city  0.080000       0.148148   
..                                 ...       ...            ...   
69     person-gender_source_concept_id  0.071429       0.133333   
70            person-race_source_value  0.081081       0.150000   
71       person-race_source_concept_id  0.034483       0.066667   
72       person-ethnicity_source_value  0.027027       0.052632   
73  person-ethnicity_source_concept_id  0.033333       0.064516   

    Corrispondenza  
0                0  
1                0  
2                0  
3                0  
4                0  
..             ...  
69               0  
70               0  
71    

## Valutazione della Corrispondenza con Precision, Recall e F1

In [37]:
# Funzioni per calcolare TP, FP e FN
def true_positives(gs_set, ann_set):
    return len(gs_set.intersection(ann_set))

def false_positives(gs_set, ann_set):
    return len(ann_set - gs_set)

def false_negatives(gs_set, ann_set):
    return len(gs_set - ann_set)

# Funzione per calcolare Precision, Recall e F1-Score
def calculate_metrics(gs_set, ann_set):
    tp = true_positives(gs_set, ann_set)
    fp = false_positives(gs_set, ann_set)
    fn = false_negatives(gs_set, ann_set)

    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return precision, recall, f1

# Funzione per confrontare gli attributi e calcolare le metriche aggregate
def compare_attributes(gs_df, ann_df, threshold=0.5):
    comparison_results = {}

    for attribute in gs_df['AttributoNomeCompleto'].unique():
        # Estrai gli insiemi di annotazioni per l'attributo corrente
        gs_annotations = set(gs_df[gs_df['AttributoNomeCompleto'] == attribute]['PrettyName'])
        ann_annotations = set(ann_df[ann_df['AttributoNomeCompleto'] == attribute]['PrettyName'])

        # Calcolo delle metriche di Precision, Recall e F1-Score
        precision, recall, f1 = calculate_metrics(gs_annotations, ann_annotations)

        # Salva i risultati
        comparison_results[attribute] = {
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        }

    return comparison_results

# Supponiamo che gs_df e ann_df siano i tuoi DataFrame contenenti i dati per GS e Annotator X
# gs_df = pd.read_csv('gold_standard.csv')
# ann_df = pd.read_csv('annotator.csv')

# Confrontiamo gli attributi
comparison_results = compare_attributes(GS, annotazioni)

# Visualizziamo i risultati
for attribute, metrics in comparison_results.items():
    print(f"Attributo: {attribute}")
    print(f"Precision: {metrics['Precision']}")
    print(f"Recall: {metrics['Recall']}")
    print(f"F1: {metrics['F1']}")
    print()

Attributo: location-location
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-location_id
Precision: 0.045454545454545456
Recall: 0.5
F1: 0.08333333333333334

Attributo: location-address_1
Precision: 0.04
Recall: 1.0
F1: 0.07692307692307693

Attributo: location-address_2
Precision: 0.038461538461538464
Recall: 1.0
F1: 0.07407407407407407

Attributo: location-city
Precision: 0.08
Recall: 1.0
F1: 0.14814814814814814

Attributo: location-state
Precision: 0.041666666666666664
Recall: 0.5
F1: 0.07692307692307693

Attributo: location-zip
Precision: 0.09090909090909091
Recall: 0.6666666666666666
F1: 0.16

Attributo: location-county
Precision: 0.10526315789473684
Recall: 1.0
F1: 0.1904761904761905

Attributo: location-country
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-location_source_value
Precision: 0.08695652173913043
Recall: 1.0
F1: 0.16

Attributo: location-latitude
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-longitude
Precision: 0
Recall: 0.0
F1: 0

Attributo: cost-cos